In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
from IPython.display import Image
from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb

# Plan for rough estimate:
   1. perennials, and grasses and non irrigated (Why google slide doess not say annuals)
    

# Function definitions

In [157]:
def filter_out_unwanted(dt_df):
    unwanted_plants = ["Almond", "Apple", "Alfalfa/Grass Hay",
                       "Apricot", "Asparagus", "Berry, Unknown",  
                       "Blueberry", "Cherry", "Grape, Juice", 
                       "Grape, Table", "Grape, Unknown", 
                       "Grape, Wine", "Hops", "Mint", 
                       "Nectarine/Peach", "Orchard, Unknown", 
                       "Pear", "Plum", "Strawberry", "Walnut"]
    
    # filter unwanted plants
    dt_df = dt_df[~(dt_df['CropTyp'].isin(unwanted_plants))]
    
    # filter non-irrigated
    """
    # These two lines can replace the following two lines
    non_irrigations = ["Unknown", "None", "None/Rill", "None/Sprinkler", 
                       "None/Sprinkler/Wheel Line", 
                       "None/Wheel Line", "Drip/None", "Center Pivot/None"]
    
    dt_df = dt_df[~(dt_df['Irrigtn'].isin(non_irrigations))]
    """
    dt_df = dt_df[~dt_df['Irrigtn'].str.contains("None")]
    dt_df = dt_df[~dt_df['Irrigtn'].str.contains("Unknown")]
    
    return dt_df
    
def initial_clean(dt):
    # remove the useles system:index column
    if ("system:index" in list(grant_2018.columns)):
        dt = dt.drop(columns=['system:index'])
    
    # Drop rows whith NA in NDVI column.
    dt = dt[dt['NDVI'].notna()]
    
    # rename the column .geo to "geo"
    dt = dt.rename(columns={".geo": "geo"})
    
    # clean non-irrigated and perrinials
    dt = filter_out_unwanted(dt)
    return (dt)

def order_by_doy(dt):
    return dt.sort_values(by='doy', axis=0, ascending=True)

# Directories

In [5]:
data_dir = "/Users/hn/Documents/01_research_data/Ag_check_point/remote_sensing/01_NDVI_TS/Grant/"

# Data Reading

In [137]:
file_names = ["Grant_2018_TS.csv"]

In [138]:
file_N = file_names[0]
grant_2018 = pd.read_csv(data_dir + file_N)

In [140]:
# Count distict values, use nunique:
pprint (grant_2018['.geo'].nunique())

# Count only non-null values, use count:
print (grant_2018['.geo'].count())

# Count total values including null values, use size attribute:
print (grant_2018['.geo'].size)

7
385
385


#### List of unique polygons

In [141]:
polygon_list = grant_2018['.geo'].unique()

# Pick one of the polygons

In [150]:
a_grant_2018_field = grant_2018[grant_2018['.geo']==polygon_list[3]]
a_grant_2018_field = order_by_doy(a_grant_2018_field)
a_grant_2018_field.loc[165, 'CropTyp'] = "Almond"

In [152]:
print(a_grant_2018_field.shape)

(57, 24)


In [158]:
print ("The data before cleaning is of size", str(a_grant_2018_field.shape))
grant_2018_cleaned = initial_clean(a_grant_2018_field)
print ("The data after cleaning is of size", str(grant_2018_cleaned.shape))

The data before cleaning is of size (57, 24)
The data after cleaning is of size (0, 23)


In [159]:
grant_2018_cleaned['Irrigtn']

Series([], Name: Irrigtn, dtype: object)

In [160]:
grant_2018_cleaned.columns

Index(['Acres', 'B2', 'B3', 'B4', 'B8', 'CovrCrp', 'CropGrp', 'CropTyp',
       'DataSrc', 'ExctAcr', 'IntlSrD', 'Irrigtn', 'LstSrvD', 'NDVI', 'Notes',
       'RtCrpTy', 'Shap_Ar', 'Shp_Lng', 'TRS', 'county', 'doy', 'year', 'geo'],
      dtype='object')